In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM


# モデル定義
def create_model_LSTM():
    model = Sequential()
    model.add(LSTM(32, input_shape=(n_samples_per_second, 14)))  # 入力の形状は（1秒ごとのサンプル数, 特徴量の数）
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='softmax'))  # バイナリ分類なので出力は1、活性化関数はsigmoid

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # 損失関数はbinary_crossentropy
    return model

# モデル定義
def create_model_NT():
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(14,)))  # 入力の形状は（特徴量の数）
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # バイナリ分類なので出力は1、活性化関数はsigmoid
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # 損失関数はbinary_crossentropy
    return model

In [6]:
#LSTMによる学習モデル（目が明いているか閉じているかを時系列的にみたモデル）

# データの読み込み
df = pd.read_csv('Raw.csv')

# 1秒ごとのデータに分割
n_samples_per_second = 256  # 256Hzのサンプリングレート
total_seconds = 10  # 全体の秒数

# ラベルの作成（5秒閉じ、5秒開け）
labels_close = [0] * 5
labels_open = [1] * 5
labels = labels_close + labels_open

# データとラベルを1秒ごとに分割
X = [df.iloc[i*n_samples_per_second:(i+1)*n_samples_per_second, :-1].values for i in range(total_seconds)]
y = labels

# 全データを一度に扱うためのリスト
X_all = []
y_all = []

# 各1秒ごとのデータセットに対してモデルを訓練
for i in range(total_seconds):
    # データの正規化
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X[i])
    X_all.append(X_scaled)

# データをnumpy配列に変換
X_all = np.array(X_all)

# 学習用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X_all, np.array(labels), test_size=0.2, random_state=0)

# モデル作成
model = create_model_LSTM()

# 学習
model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=1)

# 訓練データに対する評価
train_scores = model.evaluate(X_train, y_train, verbose=1)
print(f"Training Accuracy: {train_scores[1]*100}")

# テストデータに対する評価
test_scores = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_scores[1]*100}")

Epoch 1/50


ValueError: in user code:

    File "C:\Users\hayato yokomizo\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\hayato yokomizo\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\hayato yokomizo\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\hayato yokomizo\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\hayato yokomizo\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "C:\Users\hayato yokomizo\

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


NameError: name 'logits' is not defined

In [3]:
#時系列的に見なかった場合
# データの読み込み
df = pd.read_csv('Raw.csv')

# 特徴量とラベルを取得
X = df.drop('State', axis=1).values
y = df['State'].values

# データの正規化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 学習用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=0)

# モデル作成
model = create_model_NT()

# 学習
model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=1)

# 訓練データに対する評価
train_scores = model.evaluate(X_train, y_train, verbose=1)
print(f"Training Accuracy: {train_scores[1]*100}")

# テストデータに対する評価
test_scores = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_scores[1]*100}")

Epoch 1/50
1794/1794 [==============================] - 9s 4ms/step - loss: 0.2250 - accuracy: 0.9197
Epoch 2/50
1794/1794 [==============================] - 7s 4ms/step - loss: 0.0317 - accuracy: 0.9905
Epoch 3/50
1794/1794 [==============================] - 8s 4ms/step - loss: 0.0139 - accuracy: 0.9967
Epoch 4/50
1794/1794 [==============================] - 7s 4ms/step - loss: 0.0068 - accuracy: 0.9983
Epoch 5/50
1794/1794 [==============================] - 7s 4ms/step - loss: 0.0057 - accuracy: 0.9983
Epoch 6/50
1794/1794 [==============================] - 8s 4ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 7/50
1794/1794 [==============================] - 5s 3ms/step - loss: 0.0039 - accuracy: 0.9994
Epoch 8/50
1794/1794 [==============================] - 6s 3ms/step - loss: 0.0024 - accuracy: 0.9989
Epoch 9/50
1794/1794 [==============================] - 5s 3ms/step - loss: 9.7158e-04 - accuracy: 1.0000
Epoch 10/50
1794/1794 [==============================] - 8s 5ms/step - loss: 4